# Face recognition InsightFace

In [ ]:
%load_ext autoreload
%autoreload 2

# put system imports here
import os
import sys
from os.path import join
from glob import glob
from pathlib import Path
import random
import time

# put pip/conda installed imports here
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import imutils
import cv2 as cv
import matplotlib.pyplot as plt

# put local imports here
fp_app_root = join(Path(os.getcwd()).parent)
sys.path.append(fp_app_root)
from app.settings import app_cfg
from app.utils import file_utils, draw_utils, im_utils
from app.models.bbox import BBoxNorm

In [ ]:
os.environ['MXNET_CUDNN_AUTOTUNE_DEFAULT'] = '0'
import mxnet
import insightface
from sklearn.metrics.pairwise import cosine_similarity


## Get face image

In [ ]:
# download from https://generated.photos/
fp_dir_gan = join(app_cfg.DIR_IMAGES, 'janez_janzas')
fps_im = glob(join(fp_dir_gan, '*.jpg'))
print(len(fps_im))

In [ ]:
fa = insightface.app.FaceAnalysis()

In [ ]:
fa.prepare(-1)

In [ ]:
fps_vector = []
face_vectors = []

for fp_im in tqdm(fps_im):
  im = cv.imread(fp_im)
  im = imutils.resize(im, width=360)
  face = fa.get(im)
  if len(face):
    face = face[0]
    face_vectors.append(face.normed_embedding)
    fps_vector.append(fp_im)

## Query Image

In [ ]:
fp_im = join(app_cfg.DIR_IMAGES, 'Janez_Jansa_id-card_01.jpg')
im_query = cv.imread(fp_im)
face_query = fa.get(im_query)[0]
vector_query = face_query.normed_embedding

In [ ]:
n_results = 3
st = time.time()
sim = cosine_similarity([vector_query], face_vectors)[0]
print(time.time() - st)

In [ ]:
match_idxs = np.argpartition(sim, -n_results)[-n_results:]
match_idxs = match_idxs[np.argsort(sim[match_idxs])]
match_idxs = match_idxs[::-1]

print(match_idxs)
for i in match_idxs:
  if sim[i] > 0.5:
    print(f'Found match! --> {Path(fps_vector[i]).name}')
  print(sim[i])

In [ ]:
#ims_match = [fp_im_hanks]
ims_match = [im_query] + [cv.imread(fps_vector[i]) for i in match_idxs]

In [ ]:
montages = imutils.build_montages(ims_match, (400,400), (4,2))

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(cv.cvtColor(montages[0], cv.COLOR_BGR2RGB))